In [2]:
cd "/Users/waficel-assi/Desktop/StatsMSc/instacart-kaggle/data"

/Users/waficel-assi/Desktop/StatsMSc/instacart-kaggle/data


In [3]:
#import data science libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#import general libraries
import gc
import time


In [4]:
products = pd.read_csv('products.csv')
print(products.shape)
products.head()

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [69]:
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')

In [5]:
orders = pd.read_csv('orders.csv')
print(orders.shape)
orders.head()

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [26]:
orders_products_prior = pd.read_csv('order_products__prior.csv')
print(orders_products_prior.shape)
orders_products_prior.head(5)

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [63]:
orders_products_prior_first_10000 = orders_products_prior[orders_products_prior.order_id <= 1000000]
orders_products_prior_first_10000.shape

(9473192, 4)

**Redifine kaggle competition objective to simply predict the next product a user will purchase in latest order (eval_set = train)**

In [64]:
# MEGA DATASET
sample_join_orders = orders_products_prior_first_10000.join(orders.set_index('order_id'), how='inner')

In [65]:
print(sample_join_orders.shape)
sample_join_orders.head()

(3421083, 10)


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1,2,28985,2,1,112108,train,4,4,10,9.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,205970,prior,16,5,17,12.0
4,2,30035,5,0,178520,prior,36,1,9,7.0
5,2,17794,6,1,156122,prior,42,6,16,9.0


In [66]:
last_order_dict = (sample_join_orders
                   .groupby('order_id')
                   .agg({'add_to_cart_order':'max'})
                   .to_dict()['add_to_cart_order'])

In [67]:
sample_join_orders['last_order'] = sample_join_orders['order_id'].map(last_order_dict)
sample_join_orders['last_order_flag'] = (np.where(sample_join_orders['last_order'] == sample_join_orders['add_to_cart_order'],
                                         1,0))

In [68]:
print(sample_join_orders.shape)
sample_join_orders.head(10)

(3421083, 12)


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order,last_order_flag
1,2,28985,2,1,112108,train,4,4,10,9.0,9,0
2,2,9327,3,0,202279,prior,3,5,9,8.0,9,0
3,2,45918,4,1,205970,prior,16,5,17,12.0,9,0
4,2,30035,5,0,178520,prior,36,1,9,7.0,9,0
5,2,17794,6,1,156122,prior,42,6,16,9.0,9,0
6,2,40141,7,1,22352,prior,4,1,12,30.0,9,0
7,2,1819,8,1,142903,prior,11,2,14,30.0,9,0
8,2,43668,9,0,3107,prior,5,4,6,17.0,9,1
9,3,33754,1,1,139016,prior,14,0,19,5.0,8,0
10,3,24838,2,1,135442,prior,4,6,8,8.0,8,0


In [74]:
orders_products = sample_join_orders.merge(products, on='product_id', how='inner')
orders_products_aisles = orders_products.merge(aisles, on='aisle_id', how='inner')
orders_products_departments = orders_products_aisles.merge(departments, on='department_id', how='inner')

In [77]:
orders_products_departments.sort_values(['user_id','order_id']).head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order,last_order_flag,product_name,aisle_id,department_id,aisle,department
1774249,45690,38293,22,0,1,prior,5,4,15,28.0,32,0,Ground Turkey Breast,35,12,poultry counter,meat seafood
2013305,50089,15400,3,0,1,prior,3,3,12,21.0,6,0,Pomegranate Moisturizing Lip Balm,109,11,skin care,personal care
2078129,58222,13870,4,1,1,prior,7,1,9,20.0,4,1,Lightly Salted Baked Snap Pea Crisps,107,19,chips pretzels,snacks
2731003,125364,30142,18,0,1,train,11,4,8,14.0,21,0,Vegetable Dumpling,13,20,prepared meals,deli
873128,237822,41787,12,0,1,prior,4,4,7,29.0,12,1,Bartlett Pears,24,4,fresh fruits,produce
